In [6]:
import openai
import os
from  dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv(),override=True)

True

In [7]:
openai.key= os.getenv('OPENAI_API_KEY')

#### Basic Promt


In [8]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import AIMessage,HumanMessage,SystemMessage

In [4]:
text="""Mojo is a new programming language that bridges the gap between research and production by combining the best of Python syntax with systems programming and metaprogramming.
With Mojo, you can write portable code that’s faster than C and seamlessly inter-op with the Python ecosystem.
When we started Modular, we had no intention of building a new programming language. But as we were building our platform with the intent to unify the world’s ML/AI infrastructure, we realized that programming across the entire stack was too complicated. Plus, we were writing a lot of MLIR by hand and not having a good time.\
What we wanted was an innovative and scalable programming model that could target accelerators and other heterogeneous systems that are pervasive in the AI field. This meant a programming language with powerful compile-time metaprogramming, integration of adaptive compilation techniques, caching throughout the compilation flow, and other features that are not supported by existing languages.
And although accelerators are important, one of the most prevalent and sometimes overlooked “accelerators” is the host CPU. Nowadays, CPUs have lots of tensor-core-like accelerator blocks and other AI acceleration units, but they also serve as the “fallback” for operations that specialized accelerators don’t handle, such as data loading, pre- and post-processing, and integrations with foreign systems. So it was clear that we couldn’t lift AI with just an “accelerator language” that worked with only specific processors.
Applied AI systems need to address all these issues, and we decided there was no reason it couldn’t be done with just one language. Thus, Mojo was born.
"""

message=[
    SystemMessage(content='You are an exper copywriter with expertise in summarizing documents'),
    HumanMessage(content=f'Please provide a short and concise summary of the following text.\n TEXT:{text}'),
]



In [9]:
gpt_llm = ChatOpenAI(temperature=0,model='gpt-3.5-turbo')

In [5]:
gpt_llm.get_num_tokens(text)

320

In [6]:
summary = gpt_llm(message)

In [7]:
summary.content

"Mojo is a new programming language that combines Python syntax with systems programming and metaprogramming. It aims to bridge the gap between research and production by allowing developers to write portable code that is faster than C and seamlessly integrates with the Python ecosystem. Mojo was created to address the complexities of programming across the entire stack in the ML/AI field, with a focus on targeting accelerators and heterogeneous systems. It also recognizes the importance of the host CPU and aims to handle operations that specialized accelerators don't handle. Mojo aims to be a comprehensive language for applied AI systems."

#### Prompt Templates

In [8]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

In [9]:
template="""
Write a concise and short summary of the following text:
TEXT:{text}
Translate the summary to {language}
"""
prompt = PromptTemplate(input_variables=['text','language'],template=template)

In [10]:
gpt_llm.get_num_tokens(prompt.format(text=text,language='English'))

340

In [11]:
chain = LLMChain(llm=gpt_llm,prompt=prompt)

In [12]:
summary = chain.run({'text':text,'language':'Spanish'})

In [14]:
summary

'Mojo es un nuevo lenguaje de programación que combina lo mejor de la sintaxis de Python con la programación de sistemas y la metaprogramación, con el objetivo de unificar la infraestructura de inteligencia artificial y aprendizaje automático. Con Mojo, se puede escribir código portátil más rápido que en C y trabajar de manera integrada con el ecosistema de Python. Este lenguaje fue creado para abordar la complejidad de programar en todo el stack y para ofrecer un modelo de programación innovador y escalable que pueda dirigirse a aceleradores y otros sistemas heterogéneos en el campo de la inteligencia artificial. Además, Mojo reconoce la importancia de la CPU como un "acelerador" prevalente y a menudo pasado por alto, y busca abordar todas las necesidades de los sistemas de IA aplicada en un solo lenguaje.'

#### Summarizing using StuffDocumentChain

In [3]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [4]:
with open('files/speech.txt',encoding='utf-8') as f:
    text = f.read()
doc = [Document(page_content=text)]

In [5]:
prompt_template = '''Write a concise ans short summary of the text.TEXT:`{text}`'''
prompt=PromptTemplate(
    input_variables=['text'],
    template=prompt_template
)

In [11]:
chain = load_summarize_chain(
    llm=gpt_llm,
    prompt=prompt,
    chain_type='stuff',
    verbose=False
)
summary = chain.run(doc)

In [12]:
summary

"The text is a commencement speech given by Steve Jobs at a university graduation. He shares three stories from his life, emphasizing the importance of connecting the dots, following one's passion, and embracing the inevitability of death. He encourages the graduates to trust in themselves, stay curious, and not be afraid to take risks. The speech ends with the message to stay hungry and stay foolish, urging the graduates to continue seeking knowledge and embracing new experiences."